In [22]:
#@title Enter stuff here (get the twitter stuff from the API console)

message = "" #@param {type:"string"}

consumer_key = "" #@param {type:"string"}
consumer_secret = "" #@param {type:"string"}

bisky_username = "" #@param {type:"string"}
bisky_password = "" #@param {type:"string"}


In [ ]:
!pip install atprototools
import os,json
from typing import Dict
from atprototools import Session
from requests_oauthlib import OAuth1Session


def fetch_oauth_tokens(consumer_key: str, consumer_secret: str) -> Dict[str, str]:
    request_token_url = "https://api.twitter.com/oauth/request_token?oauth_callback=oob&x_auth_access_type=write"
    oauth = OAuth1Session(consumer_key, client_secret=consumer_secret)

    try:
        fetch_response = oauth.fetch_request_token(request_token_url)
    except ValueError:
        print("There may have been an issue with the consumer_key or consumer_secret you entered.")

    resource_owner_key = fetch_response.get("oauth_token")
    resource_owner_secret = fetch_response.get("oauth_token_secret")
    print(f"Got OAuth token: {resource_owner_key}")

    base_authorization_url = "https://api.twitter.com/oauth/authorize"
    authorization_url = oauth.authorization_url(base_authorization_url)
    print(f"Please go here and authorize: {authorization_url}")
    verifier = input("Paste the PIN here: ")

    access_token_url = "https://api.twitter.com/oauth/access_token"
    oauth = OAuth1Session(
        consumer_key,
        client_secret=consumer_secret,
        resource_owner_key=resource_owner_key,
        resource_owner_secret=resource_owner_secret,
        verifier=verifier,
    )

    
    with open('twitoauth.json', 'w', encoding='utf-8') as f:
      json.dump(oauth.fetch_access_token(access_token_url), f, ensure_ascii=False, indent=4)

def skeet_and_tweet(consumer_key: str, consumer_secret: str, access_token: str, access_token_secret: str,  bisky_username: str, bisky_password: str, payload: Dict[str, str]) -> None:
    oauth = OAuth1Session(
        consumer_key,
        client_secret=consumer_secret,
        resource_owner_key=access_token,
        resource_owner_secret=access_token_secret,
    )

    response = oauth.post("https://api.twitter.com/2/tweets", json=payload)

    if response.status_code != 201:
        raise Exception(f"Request returned an error: {response.status_code} {response.text}")

    Session(bisky_username, bisky_password).post_skoot(payload['text'])


if not (os.path.isfile("twitoauth.json")):
    fetch_oauth_tokens(consumer_key, consumer_secret)

with open('twitoauth.json', 'r', encoding='utf-8') as f:
    oauth_tokens = json.load(f)

access_token: str = oauth_tokens["oauth_token"]
access_token_secret: str = oauth_tokens["oauth_token_secret"]

skeet_and_tweet(consumer_key, consumer_secret, access_token, access_token_secret, bisky_username, bisky_password, {"text": message})

